In [8]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [9]:
MODEL_SAVE_PATH = "TTNKmodels"
MODEL_NAME = "model.ckpt"
MOVING_AVERAGE_DECAY = 0.99
# 定义神经网路相关参数|
INPUT_NODE = 33
OUTPUT_NODE = 2
LAYER1_NODE = 600

In [13]:
folder = 'TTNKdata'
_train_path=os.path.join(folder,'prepressed_test_data.csv')
train_data=pd.read_csv(_train_path,header = None)
np.array(train_data).shape
X = np.array(train_data)[:,1:34]
X = X.astype(np.float32)
    #numpy默认是float64，跟神经网络定义的时候的变量类型(float32)不符合，会出点问题
    #使用astype()改变类型

In [2]:
#获取变量
def get_weights_variable(shape, regularizer):
    weights = tf.get_variable(
        "weights", shape,dtype=tf.float32,
        initializer=tf.truncated_normal_initializer(stddev=0.1))

    # 当给出了正则化函数时，将当前变量的正则化损失加入名字为losses的集合当中
    # 这个集合不再TF自动`管理的集合列表中
    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

In [3]:
# 定义神经网路的向前传播过程
def inference(input_tensor, regularizer):  # regulaizer 正则化
    # 声明第一层神经网路的变量并完成向前传播过程
    with tf.variable_scope("layer1",reuse=tf.AUTO_REUSE):
        #  如果存在多次调用该函数，需要在第一次调用后将reuse设置为True
        weights = get_weights_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases) #Acc on val is 0.857143
        #layer1 = tf.nn.leaky_relu(tf.matmul(input_tensor, weights) + biases)#Acc on val is 0.868132
        
        # 第二层神经网络
    with tf.variable_scope("layer2",reuse = tf.AUTO_REUSE):
        weights = get_weights_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    
    #使用sotfmax转化为概率值
    layer_to_return = tf.nn.softmax(layer2)
    # 返回最后的向前传播结果
    return layer_to_return


In [19]:
def inf(X):
    # 定义输入输出placeholder
    x = tf.placeholder(tf.float32, [None,INPUT_NODE], name="x-input")
    y = inference(x,None)
    # 通过变量重命名的方式来加载模型，这样在前向传播的过程中就不需要调用求滑动平均的函数
    # 获取平均值了。这样就可以完全公用mnist_inference.py中定义的前向传播过程
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)
    
    with tf.Session() as sess:
        # tf.train.get_checkpoint_state 函数就会通过checkpoint自动找到目录中最新的模型文件名
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and  ckpt.model_checkpoint_path:
            1
            # 加载模型
            saver.restore(sess,ckpt.model_checkpoint_path)
            # 通过文件名得到模型保存的代数
            result = sess.run(y,feed_dict={x:X})
        else:
            print("No checkpoint file found!")
            return
    return result

In [23]:
Y = inf(X)

INFO:tensorflow:Restoring parameters from TTNKmodels/model.ckpt-99001


In [24]:
Y

array([[9.99026537e-01, 9.73520626e-04],
       [4.64319021e-01, 5.35681009e-01],
       [9.98116255e-01, 1.88377313e-03],
       [9.98633802e-01, 1.36615173e-03],
       [3.30143809e-01, 6.69856191e-01],
       [9.95412409e-01, 4.58758185e-03],
       [5.78950226e-01, 4.21049833e-01],
       [9.94598627e-01, 5.40134870e-03],
       [5.18939756e-02, 9.48106050e-01],
       [9.97329235e-01, 2.67082429e-03],
       [9.99035358e-01, 9.64696985e-04],
       [9.74279642e-01, 2.57203933e-02],
       [3.58062610e-03, 9.96419430e-01],
       [9.97697890e-01, 2.30211462e-03],
       [2.79325270e-03, 9.97206748e-01],
       [2.43349541e-02, 9.75665033e-01],
       [9.94903564e-01, 5.09643182e-03],
       [9.98119414e-01, 1.88061805e-03],
       [6.52600467e-01, 3.47399533e-01],
       [3.46523374e-01, 6.53476655e-01],
       [9.76181984e-01, 2.38180440e-02],
       [1.59023806e-01, 8.40976179e-01],
       [8.14687647e-03, 9.91853178e-01],
       [8.78150284e-01, 1.21849671e-01],
       [3.807412

In [31]:
is_survived = []
pas_id = 892
for y in Y:
    if y[0]<=0.5:
        is_survived.append([pas_id,0])
    else:
        is_survived.append([pas_id,1])
    pas_id+=1
is_survived = np.array(is_survived)
is_survived

array([[ 892,    1],
       [ 893,    0],
       [ 894,    1],
       [ 895,    1],
       [ 896,    0],
       [ 897,    1],
       [ 898,    1],
       [ 899,    1],
       [ 900,    0],
       [ 901,    1],
       [ 902,    1],
       [ 903,    1],
       [ 904,    0],
       [ 905,    1],
       [ 906,    0],
       [ 907,    0],
       [ 908,    1],
       [ 909,    1],
       [ 910,    1],
       [ 911,    0],
       [ 912,    1],
       [ 913,    0],
       [ 914,    0],
       [ 915,    1],
       [ 916,    0],
       [ 917,    1],
       [ 918,    0],
       [ 919,    1],
       [ 920,    1],
       [ 921,    1],
       [ 922,    1],
       [ 923,    1],
       [ 924,    1],
       [ 925,    0],
       [ 926,    0],
       [ 927,    1],
       [ 928,    0],
       [ 929,    0],
       [ 930,    1],
       [ 931,    1],
       [ 932,    1],
       [ 933,    1],
       [ 934,    1],
       [ 935,    0],
       [ 936,    0],
       [ 937,    1],
       [ 938,    1],
       [ 939,

In [38]:
folder = 'TTNKdata'
ans_save_path = os.path.join(folder,'ans_to_test_data.csv')
#通过index参数设置行索引，通过columns参数设置列索引
ans_data = pd.DataFrame(is_survived,columns=['PassengerId','Survived'])
#index参数设置为False表示不保存行索引,header设置为False表示不保存列索引
ans_data.to_csv(ans_save_path,index=False) 